In [8]:
import json
import os
from collections import defaultdict
from datetime import datetime, timedelta
from glob import glob
from math import asin, cos, radians, sin, sqrt
from numbers import Real
from typing import Any, Iterator, Tuple, Union
from sshtunnel import SSHTunnelForwarder

import paramiko
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import sshtunnel
import MySQLdb

from numpy import positive
from pyproj import Transformer, transform
from shapely.geometry import Point, Polygon
from sqlalchemy import (
    Connection,
    CursorResult,
    Engine,
    Select,
    TextClause,
    create_engine,
    func,
    select,
)
from sqlalchemy.sql.expression import text
from pyopensky.trino import Trino

In [2]:
timelike = Union[str, Real, datetime, pd.Timestamp]

def to_datetime(time: timelike) -> pd.Timestamp:
    if isinstance(time, str):
        time = pd.Timestamp(time, tz="utc")
    elif isinstance(time, datetime):
        time = pd.to_datetime(time, utc=True)
    elif isinstance(time, Real):
        time = pd.Timestamp(float(time), unit="s", tz="utc")
    return time

In [4]:
LAT_MIN, LAT_MAX = 50.896393, 50.967115
LON_MIN, LON_MAX = 6.919968, 7.005756
ALT_MIN, ALT_MAX = 0, 750 # update from 700 m to 750 m, in line with CTR limit at 2500 ft plus margin (and accounting for Geoid Height)

TIME_BETWEEN_TRAJS = 30

# SERA.5005(f)(1) criteria
CONGESTED_ALERT_DISTANCE_M = 600      # alert distance wrt obstacles (should be 600)
CONGESTED_ALERT_DELTA_HEIGHT_M = 300   # delta height (should be 300)

# SERA.5005(f)(2) criteria
NONCONGESTED_ALERT_DISTANCE_M = 150 
NONCONGESTED_ALERT_DELTA_HEIGHT_M = 150  

CPA_MARGIN_M = 20 # allowance for lateral distance to obstacle
DIP_MARGIN_M = 20 # allowance for dip below minimum height (45m corresponds to GVA = 2)
N_MIN = 5

GEOID_HEIGHT_M = 47  # geoid height for Cologne

sshtunnel.SSH_TIMEOUT = 15.0
sshtunnel.TUNNEL_TIMEOUT = 15.0

# Setup OpenSky Network Trino credentials

In [5]:
OSN_secrets_json = './trino_secrets.json'
MYSQL_secrets_json = './mysql_secrets.json'

with open(OSN_secrets_json) as OSN_secrets:
  OSN_creds = json.load(OSN_secrets)

with open(MYSQL_secrets_json) as MYSQL_secrets:
  MYSQL_creds = json.load(MYSQL_secrets)

os.environ['OPENSKY_USERNAME'] = OSN_creds['OPENSKY_USERNAME']
os.environ['OPENSKY_PASSWORD'] = OSN_creds['OPENSKY_PASSWORD']

# Test connection with basic SQL query

In [5]:


trino = Trino()

query = "select * from state_vectors_data4 limit 5"

df = trino.query(query, cached = False)

FINISHED: : 24.6% [00:00, 211%/s]
DOWNLOAD: 5.00lines [00:00, 100lines/s]


# Send queries and merge results

In [6]:
start_str = "01/09/24"
end_str = "07/09/24"

start = datetime.strptime(start_str, '%d/%m/%y')
end = datetime.strptime(end_str, '%d/%m/%y')

# Modify start_date to be 00:00:00
start = start.replace(hour=0, minute=0, second=1, microsecond=0)
# Modify end_date to be 23:59:59
end = end.replace(hour=23, minute=59, second=59, microsecond=999999)

start_time = int(start.timestamp())
start_hour = start_time - (start_time % 3600)
end_time = int(end.timestamp())
end_hour = end_time - (end_time % 3600)

In [9]:
svdata4_query = (
        f"SELECT * FROM state_vectors_data4"
        f" WHERE icao24 LIKE '%'"
        f" AND time >= {start_time} AND time <= {end_time}"
        f" AND hour >= {start_hour} AND hour <= {end_hour}"
        f" AND lat >= {LAT_MIN} AND lat <= {LAT_MAX}"
        f" AND lon>= {LON_MIN} AND lon <= {LON_MAX}"
        f" AND geoaltitude >= {ALT_MIN} AND geoaltitude <= {ALT_MAX}"
        f" ORDER BY time"
    )

trino = Trino()

svdata4_df = trino.query(
    svdata4_query,
    cached=False,
    compress=True,
)

FINISHED: : 100% [00:01, 81.8%/s]
DOWNLOAD: 8.04klines [00:00, 27.8klines/s]


In [10]:
icao_list = svdata4_df.icao24.unique()
icao24_str = ', '.join(f"'{item}'" for item in icao_list)

ops_sts_query = (
    f"SELECT icao24, mintime, maxtime, nacv, systemdesignassurance, version, positionnac, geometricverticalaccuracy, sourceintegritylevel, barometricaltitudeintegritycode  FROM operational_status_data4"
    f" WHERE icao24 IN ({icao24_str})"
    f" AND mintime >= {start_time} AND maxtime <= {end_time}"
    f" AND hour >= {start_hour} AND hour <= {end_hour}"
    f" ORDER by mintime"
)

print('Connecting to OSN database...')
trino = Trino()
ops_sts_df = trino.query(
    ops_sts_query,
    cached=False,
)

ops_sts_df['time'] = ops_sts_df['mintime'].astype('int64')

Connecting to OSN database...


FINISHING: : 100% [00:02, 39.2%/s]
DOWNLOAD: 543klines [00:24, 22.5klines/s]


In [11]:
# Initialize an empty DataFrame to hold the results
merged_df = pd.DataFrame()

# Loop over each unique 'icao24' in both dataframes
unique_icao24s = pd.concat([svdata4_df['icao24'], ops_sts_df['icao24']]).unique()

for icao24 in unique_icao24s:
    # Filter each dataframe by 'icao24'
    sub_df1 = svdata4_df[svdata4_df['icao24'] == icao24]
    sub_df2 = ops_sts_df[ops_sts_df['icao24'] == icao24]

    # Ensure both sub-dataframes are sorted by 'time'
    sub_df1 = sub_df1.sort_values('time')
    sub_df2 = sub_df2.sort_values('time')

    # Perform merge_asof on the filtered and sorted dataframes
    merged_sub_df = pd.merge_asof(sub_df1, sub_df2, on='time', by='icao24', direction='backward')
    
    # Append the result to the main dataframe
    merged_df = pd.concat([merged_df, merged_sub_df], ignore_index=True)


In [12]:
posdata4_query = (
    f"SELECT mintime, icao24, nic  FROM position_data4"
    f" WHERE icao24 IN ({icao24_str})"
    f" AND lat >= {LAT_MIN} AND lat <= {LAT_MAX}"
    f" AND lon>= {LON_MIN} AND lon <= {LON_MAX}"
    f" AND mintime >= {start_time} AND maxtime <= {end_time}"
    f" AND hour >= {start_hour} AND hour <= {end_hour}"
    f" ORDER by mintime"
)

print('Connecting to OSN database...')
trino = Trino()
posdata4_df = trino.query(
    posdata4_query,
    cached=False,
)

posdata4_df['time'] = posdata4_df['mintime'].astype('int64')

Connecting to OSN database...


FINISHED: : 100% [00:00, 370%/s]
DOWNLOAD: 23.2klines [00:00, 288klines/s]


In [13]:
# Initialize an empty DataFrame to hold the results
final_df = pd.DataFrame()

for icao24 in unique_icao24s:
    # Filter each dataframe by 'icao24'
    sub_df1 = merged_df[merged_df['icao24'] == icao24]
    sub_df2 = posdata4_df[posdata4_df['icao24'] == icao24]

    # Ensure both sub-dataframes are sorted by 'time'
    sub_df1 = sub_df1.sort_values('time')
    sub_df2 = sub_df2.sort_values('time')

    # Perform merge_asof on the filtered and sorted dataframes
    merged_sub_df = pd.merge_asof(sub_df1, sub_df2, on='time', by='icao24', direction='backward')
    
    # Append the result to the main dataframe
    final_df = pd.concat([final_df, merged_sub_df], ignore_index=True)

final_df = final_df.drop(columns=['hour', 'mintime_x', 'maxtime', 'mintime_y'])


# Add DEM ground elevation information

In [14]:
crs_transformer = Transformer.from_crs(4326, 25832, always_xy = True) # Transformer from WGS-84 to ETRS89-LAEA (3035 for EU-DEM v1.1, 25832 for LAS DEM)

def transform_coords(lon, lat):
    return crs_transformer.transform(lon, lat)

In [15]:
final_df['gnd_elev'] = np.nan

dem_src = rasterio.open('./resources/Cologne_DEM_merged_from_LAS_25x25.tif')

final_df['etrs89_x'], final_df['etrs89_y'] = zip(*final_df.apply(lambda row: transform_coords(row['lon'], row['lat']), axis=1))


In [16]:
def get_elevation(x, y, dem):
    row, col = dem.index(x, y)
    return dem.read(1)[row, col]

final_df['gnd_elev'] = final_df.apply(lambda row: get_elevation(row['etrs89_x'], row['etrs89_y'], dem_src), axis=1)

# Add population density

In [17]:
final_df['pop_density'] = np.nan

pop_src = rasterio.open('./resources/GHS_POP_E2020_GLOBE_R2023A_4326_3ss_V1_0_Cologne.tif')

def get_population(lon, lat, pop_src):
    row, col = pop_src.index(lon, lat)
    return pop_src.read(1)[row, col]

final_df['pop_density'] = final_df.apply(lambda row: get_population(row['lon'], row['lat'], pop_src), axis=1)


# Process df with distance information

In [19]:
def haversine(pt1, pt2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    Returned units are in metres. Differs slightly from PostGIS geography
    distance, which uses a spheroid, rather than a sphere.
    """

    lat1, lon1 = pt1[0], pt1[1]
    lat2, lon2 = pt2[0], pt2[1]

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2.0) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2.0) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371000  # Radius of earth in m
    return c * r

In [23]:
final_df["prev_time"] = final_df.time.shift()
final_df['closest_obst_name'] = ''
final_df['inf_flt'] = False
final_df['inf_pt'] = False
final_df['gnd_inf_flt'] = False
final_df['gnd_inf_pt'] = False
final_df['min_hgt'] = np.nan
final_df['congested'] = final_df['pop_density'] > 2

map_time_traj = defaultdict(dict)

for icao, sfinal_df in final_df.groupby("icao24"):
    map_time_traj[icao][sfinal_df.iloc[0]["time"]] = icao + "_1"
    n_traj = 1
    for i in range(1, sfinal_df.shape[0]):
        time = sfinal_df.iloc[i]["time"]
        diff = abs(int(time) - int(sfinal_df.iloc[i]["prev_time"]))
        if diff > TIME_BETWEEN_TRAJS:
            n_traj += 1
        map_time_traj[icao][time] = icao + "_" + str(int(n_traj))

final_df['ref'] = final_df.apply(lambda x: map_time_traj[x.icao24][x.time], axis=1) + '_' + final_df.time.apply(lambda x: pd.to_datetime(x, unit='s').strftime("%d%m%y"))

# Add a distance column and compute cumulative along-track distance for each flight
final_df['dist'] = 0.0
for flight in final_df.ref.unique():
  first = True
  current = final_df[final_df['ref'].isin([flight])] # gets the trajectory of the current flight
  for row in current.itertuples():
    if not(first):
      current_pt = (float(row.lat), float(row.lon))
      delta_dist = haversine(previous_pt, current_pt)
      final_df.loc[row[0],'dist'] = previous_dist + delta_dist
    previous_pt = (float(row.lat), float(row.lon))
    previous_dist = final_df.loc[row[0],'dist']
    first = False


# Load obstacle information and check min height

In [22]:
path_to_obstacles_json = './resources/LAC_obstacles_v1.csv'

obs_df = pd.read_csv(path_to_obstacles_json)

obs_df.rename(columns={'h': 'height_m'}, inplace=True)

obs_df = obs_df.sort_values(by=['height_m']) # sort obstacles by incresing height, to avoid that the min_hgt profil is wrong if a shorter obstacle comes after a taller one, in case the aircraft is within two obstacles clearance areas


In [24]:
def update_closest_obstacle_xy(final_df, obstacles_df):

    # Extract the coordinates and obstacle heights
    final_coords = final_df[['etrs89_x', 'etrs89_y']].to_numpy()
    obstacle_coords = obstacles_df[['x', 'y']].to_numpy()
    obstacle_heights = obstacles_df['height_m'].to_numpy()
    obstacle_names = obstacles_df['LAC_Name'].to_numpy()
    obstacle_ground_elevs = obstacles_df['gnd_elev'].to_numpy()

    # Iterate over each point in final_df and calculate distances
    for i, (x_f, y_f) in enumerate(final_coords):

        radius = (CONGESTED_ALERT_DISTANCE_M if final_df.at[i, 'congested'] else NONCONGESTED_ALERT_DISTANCE_M)

        # Convert radius to a squared radius for comparison with squared distances (faster than computing square roots)
        squared_radius = radius ** 2

        # Calculate squared Euclidean distances to all obstacles
        distances_sq = (obstacle_coords[:, 0] - x_f) ** 2 + (obstacle_coords[:, 1] - y_f) ** 2
        
        # Filter obstacles within the radius
        within_radius = distances_sq <= squared_radius
        
        # If there are obstacles within the radius, find the tallest one
        if np.any(within_radius):
            # Get the indices of obstacles within the radius
            obstacles_in_radius_idx = np.where(within_radius)[0]
            
            # Find the index of the tallest obstacle within the radius
            tallest_idx = obstacles_in_radius_idx[np.argmax(obstacle_heights[obstacles_in_radius_idx])]
            
            # Update final_df with the tallest obstacle details
            tallest_obstacle_name = obstacle_names[tallest_idx]
            tallest_obstacle_height = obstacle_heights[tallest_idx]
            tallest_obstacle_elev = obstacle_ground_elevs[tallest_idx]
            
            final_df.at[i, 'closest_obst_name'] = tallest_obstacle_name

            final_df.at[i, 'min_hgt'] = (
                GEOID_HEIGHT_M + np.float32(tallest_obstacle_elev) +
                np.float32(tallest_obstacle_height) + 
                (CONGESTED_ALERT_DELTA_HEIGHT_M if final_df.at[i, 'congested'] else NONCONGESTED_ALERT_DELTA_HEIGHT_M)
            )
        else:
            final_df.at[i, 'closest_obst_name'] = 'ground'
            final_df.at[i, 'min_hgt'] = (CONGESTED_ALERT_DELTA_HEIGHT_M if final_df.at[i, 'congested'] else NONCONGESTED_ALERT_DELTA_HEIGHT_M)

    return final_df


In [25]:
final_df = update_closest_obstacle_xy(final_df, obs_df)

final_df['dip'] = final_df['min_hgt'] - final_df['geoaltitude']

# Add infraction information

In [27]:
# Rule for 'inf_pt'
final_df['inf_pt'] = final_df.apply(
    lambda row: 
        True if row['dip'] > 0 and row['closest_obst_name'] != 'ground' 
    else 
        False,
    axis=1
)

# Rule for 'gnd_inf_pt'
final_df['gnd_inf_pt'] = final_df.apply(
    lambda row: True if (row['dip'] > 0 and row['closest_obst_name'] == 'ground') else False,
    axis=1
)

# Group by 'ref' and update 'inf_flt' based on 'inf_pt'
final_df['inf_flt'] = final_df.groupby('ref')['inf_pt'].transform('any')

# Group by 'ref' and update 'gnd_inf_flt' based on 'gnd_inf_pt'
final_df['gnd_inf_flt'] = final_df.groupby('ref')['gnd_inf_pt'].transform('any')

final_df = final_df.drop(columns=['serials', 'nacv'])


In [29]:
final_df[final_df['gnd_inf_pt']].callsign.unique()

array([None, 'CHX3    ', 'HUMMEL6 ', 'HUMMEL1 ', 'CHX09   '], dtype=object)

# Create infraction tables

In [32]:
inf_pt_df = final_df[final_df.inf_pt].copy()

#lat, lon = obs_df.loc[obs_df['name'] == 'Ringturm', ['lat', 'lon']].values[0]

inf_pt_df['dist_to_obs'] = np.nan

inf_pt_df['dist_to_obs'] = inf_pt_df.apply(
    lambda x: haversine(
            (obs_df.loc[obs_df['LAC_Name'] == x['closest_obst_name'], 'lat'].iloc[0],
            obs_df.loc[obs_df['LAC_Name'] == x['closest_obst_name'], 'lon'].iloc[0]),
            (x['lat'], x['lon'])
    ) if not (x['closest_obst_name'] == 'ground') else np.nan, 
    axis=1
)

inf_pt_df['time'] = pd.to_datetime(inf_pt_df['time'], unit='s')
inf_pt_df = inf_pt_df.sort_values(by=['ref', 'closest_obst_name', 'time'])

inf_pt_df['time_diff'] = inf_pt_df.groupby(['ref', 'closest_obst_name'])['time'].diff()
inf_pt_df['group'] = (inf_pt_df['time_diff'] >= pd.Timedelta(seconds=30)).cumsum()

inf_grouped = inf_pt_df.groupby(['ref', 'closest_obst_name', 'group'])

inf_min_dist = inf_grouped.apply(lambda x: x.loc[x['dist_to_obs'].idxmin()]).reset_index(drop=True)
inf_max_dip = inf_grouped['dip'].max().reset_index()

group_size = inf_grouped.size().reset_index(name='n')

inf_result = inf_min_dist[['icao24', 'callsign', 'group', 'ref', 'closest_obst_name', 'time', 'lat', 'lon', 'dist_to_obs', 'congested', 'pop_density']].copy()
inf_result = inf_result.merge(inf_max_dip, on=['ref', 'closest_obst_name', 'group'])
inf_result = inf_result.merge(group_size, on=['ref', 'closest_obst_name', 'group'])

inf_result.rename(columns={'dist_to_obs': 'cpa', 'dip': 'dip_max', 'closest_obst_name': 'obstacle_name'}, inplace=True)

inf_result['entry_count'] = inf_result.groupby('ref').cumcount()
inf_result['inf_ref'] = inf_result['ref'].astype(str) + '_' + inf_result['entry_count'].astype(str)

inf_result['url'] = inf_result.apply(lambda row: "https://globe.adsbexchange.com/?icao=%s&lat=50.928&lon=6.947&zoom=13.2&showTrace=%s&timestamp=%s" % (
    row['icao24'],
    row['time'].strftime('%Y-%m-%d'),
    str(int(row['time'].timestamp()))
), axis=1)

inf_result = inf_result.reset_index(drop=True)

inf_result = inf_result.drop(columns=['entry_count', 'group'])

In [33]:
inf_result

,icao24,callsign,ref,obstacle_name,time,lat,lon,cpa,congested,pop_density,dip_max,n,inf_ref,url
0,398338,FHAZY,398338_1_050924,AREO Turm,2024-09-05 18:54:02,50.914352,6.966965,395.997842,True,25.732103,19.700000,17,398338_1_050924_0,https://globe.adsbexchange.com/?icao=398338&la...
1,398338,FHAZY,398338_1_050924,Colonius FernsehturmColonius Fernsehturm,2024-09-05 18:52:07,50.947083,6.932447,36.547277,True,3.437461,88.359985,30,398338_1_050924_1,https://globe.adsbexchange.com/?icao=398338&la...
2,398338,FHAZY,398338_1_050924,Krohstr. 4 (Bayenthal),2024-09-05 18:54:23,50.908813,6.974240,207.934854,True,36.879350,35.500002,30,398338_1_050924_2,https://globe.adsbexchange.com/?icao=398338&la...
3,398338,FHAZY,398338_1_050924,Rodenkirchener Bruecke 1,2024-09-05 18:55:04,50.899017,6.988864,321.243882,True,4.585226,20.599999,18,398338_1_050924_3,https://globe.adsbexchange.com/?icao=398338&la...
4,3c6dd4,EWG363,3c6dd4_1_020924,An der Schanz 1,2024-09-02 13:42:51,50.965949,6.983261,489.420249,True,28.407547,73.190000,1,3c6dd4_1_020924_0,https://globe.adsbexchange.com/?icao=3c6dd4&la...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,4866f0,PHIMM,4866f0_1_070924,Koelnturm (Mediapark),2024-09-07 08:50:58,50.947632,6.942023,89.071483,True,15.683982,22.719998,18,4866f0_1_070924_3,https://globe.adsbexchange.com/?icao=4866f0&la...
169,4866f0,PHIMM,4866f0_1_070924,Pollonius,2024-09-07 08:53:56,50.926300,7.005120,522.331201,True,3.485078,5.539999,1,4866f0_1_070924_4,https://globe.adsbexchange.com/?icao=4866f0&la...
170,4866f0,PHIMM,4866f0_2_070924,Koelner Dom,2024-09-07 10:44:50,50.937366,6.955490,470.187289,True,44.071258,41.200001,15,4866f0_2_070924_0,https://globe.adsbexchange.com/?icao=4866f0&la...
171,4866f0,PHIMM,4866f0_2_070924,Koelnturm (Mediapark),2024-09-07 10:44:06,50.948212,6.942596,37.229503,True,16.949003,22.719998,27,4866f0_2_070924_1,https://globe.adsbexchange.com/?icao=4866f0&la...


Ground infractions

In [34]:
gnd_inf_pt_df = final_df[final_df.gnd_inf_pt & (final_df.dip >= 0) & (final_df.closest_obst_name == 'ground')].copy()

gnd_inf_pt_df['time'] = pd.to_datetime(gnd_inf_pt_df['time'], unit='s')
gnd_inf_pt_df = gnd_inf_pt_df.sort_values(by=['ref', 'time'])

gnd_inf_pt_df['time_diff'] = gnd_inf_pt_df.groupby(['ref'])['time'].diff()
gnd_inf_pt_df['group'] = (gnd_inf_pt_df['time_diff'] >= pd.Timedelta(seconds=30)).cumsum()

gnd_inf_grouped = gnd_inf_pt_df.groupby(['ref', 'group'])

gnd_inf_max_dip = gnd_inf_grouped.apply(lambda x: x.loc[x['dip'].idxmax()]).reset_index(drop=True)

group_size = gnd_inf_grouped.size().reset_index(name='n')

gnd_inf_result = gnd_inf_max_dip[['icao24', 'callsign', 'group', 'ref', 'closest_obst_name', 'time', 'lat', 'lon', 'dip', 'congested', 'pop_density']].copy()

gnd_inf_result = gnd_inf_result.merge(group_size, on=['ref', 'group'])

gnd_inf_result.rename(columns={'dip': 'dip_max', 'closest_obst_name': 'obstacle_name'}, inplace=True)

gnd_inf_result['entry_count'] = gnd_inf_result.groupby('ref').cumcount()
gnd_inf_result['inf_ref'] = gnd_inf_result['ref'].astype(str) + '_' + 'gnd_' + gnd_inf_result['entry_count'].astype(str)

gnd_inf_result['url'] = gnd_inf_result.apply(lambda row: "https://globe.adsbexchange.com/?icao=%s&lat=50.928&lon=6.947&zoom=13.2&showTrace=%s&timestamp=%s" % (
    row['icao24'],
    row['time'].strftime('%Y-%m-%d'),
    str(int(row['time'].timestamp()))
), axis=1)

gnd_inf_result = gnd_inf_result.reset_index(drop=True)

gnd_inf_result = gnd_inf_result.drop(columns=['entry_count', 'group'])



In [39]:
final_df.columns

Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'vertrate',
       'callsign', 'onground', 'alert', 'spi', 'squawk', 'baroaltitude',
       'geoaltitude', 'lastposupdate', 'lastcontact', 'systemdesignassurance',
       'version', 'positionnac', 'geometricverticalaccuracy',
       'sourceintegritylevel', 'barometricaltitudeintegritycode', 'nic',
       'gnd_elev', 'etrs89_x', 'etrs89_y', 'pop_density', 'prev_time',
       'closest_obst_name', 'inf_flt', 'inf_pt', 'gnd_inf_flt', 'gnd_inf_pt',
       'min_hgt', 'congested', 'ref', 'dist', 'dip'],
      dtype='object')

In [37]:
gnd_inf_result.callsign.unique()

array(['HUMMEL1 ', 'HUMMEL6 ', None, 'CHX3    ', 'CHX09   '], dtype=object)

# Upload data to MySQL server

In [41]:
ed25519_key = paramiko.Ed25519Key(filename='/home/dimitri/.ssh/id_ed25519')

# Set up the SSH tunnel with the RSA key
with SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS'], 22),
    ssh_username=MYSQL_creds['SSH_USERNAME'],
    ssh_pkey=ed25519_key,  # Use the loaded RSA key
    remote_bind_address=(MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT']),
    allow_agent=False,
) as tunnel:
    
    print('connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['TEST_DATABASE_NAME']

    engine = create_engine(engstr)

    print('step 1')
    
    final_df.to_sql(con=engine, name = MYSQL_creds['MAIN_TEST_TABLE_NAME'], if_exists='append')
    inf_result.to_sql(con=engine, name = MYSQL_creds['INF_TEST_TABLE_NAME'], if_exists='append')
    gnd_inf_result.to_sql(con=engine, name = MYSQL_creds['INF_TEST_TABLE_NAME'], if_exists='append')

    print("Done")

connected
step 1
Done


# Load Distant data for tests

In [221]:
with sshtunnel.SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS']),
    ssh_username = MYSQL_creds['SSH_USERNAME'], ssh_password = MYSQL_creds['SSH_PASSWORD'],
    remote_bind_address = (MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT'])
) as tunnel:
    print('connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['TEST_DATABASE_NAME']

    engine = create_engine(engstr)

    sql_query = """
    SELECT ref, lat, lon 
    FROM main_data_test
    WHERE positionnac >= 8
    """
    
    distant_df = pd.read_sql(sql_query, con=engine)

connected


In [ ]:
SELECT 
    COLUMN_NAME, 
    IS_NULLABLE 
FROM 
    INFORMATION_SCHEMA.COLUMNS 
WHERE 
    TABLE_NAME = 'main_data_test' 
    AND TABLE_SCHEMA = 'testdatabase';